# Australian Marriage Law Survey Data Cleaning

**According to wikipedia** : The Australian Marriage Law Postal Survey was a national survey designed to gauge support for legalising same-sex marriage in Australia. The survey was held via the postal service between 12 September and 7 November 2017. Unlike voting in elections and referendums, which is compulsory in Australia, responding to the survey was voluntary. 

In this notebook, clean the raw data from the **Australian Marriage Law Postal Survey**. The data can be found online on the Australian Bureau of Statistics' website.

*The data is taken from an Excel file, which is formatted in a "non_tidy" way* which makes it hard to work with - a great deal of effort goes into preparing the data for analysis before visualization.
   
*Note that this notebook is really part of a greater whole - we clean the source data in this notebook, but there is an additional notebook visualizes the data, and another one that normalizes the data - and then uses SQLalchemy to import the results into SMSS (microsoft sql server management tool). On top of that, we create an interactive dashboard that looks at the results in Power BI*

In [2]:
# Importing necessary Packages (Pandas and Numpy)
import pandas as pd
import numpy as np

## Data Cleaning

The original Excel Files are : 
1. "Australian_marriage_law_participation.xls" **(AMLP)**
2. "australian_marriage_law_survey_response.xls" **(AMLSR)**

**AMLP** contains

**AMLSR** contains

We first run through the structure of both files by showing a series of cleaning code for **AMLP** - afterwards, we abstract the cleaning process into a function and then use it on **AMLSR** as well

### Step 1 : Understanding the File Structure (AMLP)

* We display code that showcases the original data's messiness
* This will provide context for what the cleaning code does

In [14]:
# Define file path
file_name = "australian_marriage_law_participation.xls"

# read in the data
df = pd.read_excel(file_name, 
                      sheet_name='Table 4', # obtain data from table 4 in the file
                      usecols="A:S", # the relevant columns are "A:S"
                      skiprows=6, # skip 7 rows so we start from row 8
                      names=["area", 
                             'statistic', 
                             "18-19 years",
                             "20-24 years",
                             "25-29 years",
                             "30-34 years",
                             "35-39 years",
                             "40-44 years",
                             "45-49 years",
                             "50-54 years",
                             "55-59 years",
                             "60-64 years",
                             "65-69 years",
                             "70-74 years",
                             "75-79 years",
                             "80-84 years",
                             "85 years and over",
                             "Age or gender ndf",
                             "Total Persons"] # input a column name
                     )

In [15]:
df.head(5)

,area,statistic,18-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85 years and over,Age or gender ndf,Total Persons
0,Banks,Total participants,2355.0,6361.0,5933.0,5913.0,6336.0,6726.0,7354.0,7455.0,7752.0,6801.0,5944.0,4906.0,3721.0,3024.0,3265.0,480.0,84326.0
1,NaN,Eligible participants,2945.0,8319.0,7995.0,8258.0,8559.0,8750.0,9290.0,9258.0,9347.0,8017.0,6777.0,5501.0,4165.0,3474.0,4027.0,572.0,105254.0
2,NaN,Participation rate (%),80.0,76.5,74.2,71.6,74.0,76.9,79.2,80.5,82.9,84.8,87.7,89.2,89.3,87.0,81.1,83.9,80.1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barton,Total participants,2038.0,6252.0,6638.0,7531.0,7298.0,7520.0,7849.0,7431.0,6740.0,6165.0,5650.0,4711.0,3754.0,2848.0,2542.0,396.0,85363.0


In [16]:
df.tail(3)

,area,statistic,18-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85 years and over,Age or gender ndf,Total Persons
649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
651,© Commonwealth of Australia 2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Step 2 : Cleaning the Excel File (AMLP)

In [17]:
# Step 1 : Remove the unnecessary data that resides at the bottom of the spreadsheet
df = df.drop(range(643, 652))
df.tail()

,area,statistic,18-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85 years and over,Age or gender ndf,Total Persons
638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640,Total,Total participants,325493.0,920499.0,930963.0,971850.0,970884.0,1010691.0,1112874.0,1087920.0,1136598.0,1059087.0,991429.0,814363.0,564235.0,378284.0,349107.0,103643.0,12727920.0
641,NaN,Eligible participants,416181.0,1271732.0,1295239.0,1342005.0,1322980.0,1339723.0,1422089.0,1345765.0,1360321.0,1229770.0,1121651.0,909358.0,634046.0,435719.0,434548.0,125053.0,16006180.0
642,NaN,Participation rate (%),78.2,72.4,71.9,72.4,73.4,75.4,78.3,80.8,83.6,86.1,88.4,89.6,89.0,86.8,80.3,82.9,79.5


In [18]:
# Step 2 : Remove rows where it contains the observation "Divison" and "Australia" itself. 
#          These are summary statistics.
df1 = df[~df['area'].str.contains("Division|Australia$", na=False)].reset_index()
df1.tail(3)

,index,area,statistic,18-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85 years and over,Age or gender ndf,Total Persons
632,640,Total,Total participants,325493.0,920499.0,930963.0,971850.0,970884.0,1010691.0,1112874.0,1087920.0,1136598.0,1059087.0,991429.0,814363.0,564235.0,378284.0,349107.0,103643.0,12727920.0
633,641,NaN,Eligible participants,416181.0,1271732.0,1295239.0,1342005.0,1322980.0,1339723.0,1422089.0,1345765.0,1360321.0,1229770.0,1121651.0,909358.0,634046.0,435719.0,434548.0,125053.0,16006180.0
634,642,NaN,Participation rate (%),78.2,72.4,71.9,72.4,73.4,75.4,78.3,80.8,83.6,86.1,88.4,89.6,89.0,86.8,80.3,82.9,79.5


In [19]:
# Step 3 : Drop rows where we see total participant data - we do not need this
df1 = df1.drop(range(632,635)).drop('index', axis=1)
df1.tail()

,area,statistic,18-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85 years and over,Age or gender ndf,Total Persons
627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628,Australian Capital Territory (Total),Total participants,6975.0,19818.0,21202.0,22631.0,22115.0,20781.0,21296.0,19196.0,19114.0,16660.0,15487.0,11943.0,7811.0,5081.0,4620.0,2783.0,237513.0
629,NaN,Eligible participants,8605.0,25566.0,27266.0,28902.0,28033.0,26085.0,25886.0,22954.0,22116.0,18835.0,17172.0,13133.0,8673.0,5802.0,5787.0,3293.0,288108.0
630,NaN,Participation rate (%),81.1,77.5,77.8,78.3,78.9,79.7,82.3,83.6,86.4,88.5,90.2,90.9,90.1,87.6,79.8,84.5,82.4
631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Step 4 : Remove rows where it contains only NaN. These are a quirk of the Excel Spreadsheet
df2 = df1.dropna(how='all')
df2 = df2.reset_index().drop('index', axis=1)
df2.tail()

,area,statistic,18-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85 years and over,Age or gender ndf,Total Persons
469,NaN,Eligible participants,3912.0,12924.0,14354.0,16034.0,15916.0,14257.0,12933.0,10566.0,9635.0,8403.0,7629.0,5773.0,3733.0,2506.0,2359.0,1543.0,142477.0
470,NaN,Participation rate (%),79.7,76.9,76.9,77.3,78.5,79.6,81.9,83.0,86.1,88.6,90.0,90.6,90.4,86.8,80.2,83.7,81.6
471,Australian Capital Territory (Total),Total participants,6975.0,19818.0,21202.0,22631.0,22115.0,20781.0,21296.0,19196.0,19114.0,16660.0,15487.0,11943.0,7811.0,5081.0,4620.0,2783.0,237513.0
472,NaN,Eligible participants,8605.0,25566.0,27266.0,28902.0,28033.0,26085.0,25886.0,22954.0,22116.0,18835.0,17172.0,13133.0,8673.0,5802.0,5787.0,3293.0,288108.0
473,NaN,Participation rate (%),81.1,77.5,77.8,78.3,78.9,79.7,82.3,83.6,86.4,88.5,90.2,90.9,90.1,87.6,79.8,84.5,82.4


In [21]:
# Step 5 : Notice how there are always 2 NaN rows in the 'area' column
#          This is a quirk within the excel spreadsheet. We want to forward fill
df2['area'] = df2['area'].fillna(method='ffill')

### Step 3 : Final Touches (AMLP)

* The dataframe as of now follows a format which is difficult for data visualization and data modelling
* We will attempt to fix this by "melting" the dataframe
* and then cleaning it up

In [22]:
# Now, we need to "melt" our data
df3 = pd.melt(df2, id_vars=['area', 'statistic'], var_name = 'age_category', value_name='quantity')
df3.head(5)

,area,statistic,age_category,quantity
0,Banks,Total participants,18-19 years,2355.0
1,Banks,Eligible participants,18-19 years,2945.0
2,Banks,Participation rate (%),18-19 years,80.0
3,Barton,Total participants,18-19 years,2038.0
4,Barton,Eligible participants,18-19 years,2587.0


In [23]:
# remove unnecessary totals, clean strings
df3 = df3[~df3['area'].str.contains("(Total)", na=False)].reset_index().drop('index', axis=1)
df3['area'] = df3['area'].str.replace(r"\([^)]*\)", "", regex=True)
df3.tail()

<ipython-input-23-4193910d2aea>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df3 = df3[~df3['area'].str.contains("(Total)", na=False)].reset_index().drop('index', axis=1)


,area,statistic,age_category,quantity
7645,Canberra,Eligible participants,Total Persons,145631.0
7646,Canberra,Participation rate (%),Total Persons,83.2
7647,Fenner,Total participants,Total Persons,116281.0
7648,Fenner,Eligible participants,Total Persons,142477.0
7649,Fenner,Participation rate (%),Total Persons,81.6


In [24]:
# We also want to create a new columns which identify the state of the area - Australia has 5 states

# Obtain Crosswalk
cross_walk = pd.read_csv('division_mapping.csv')
cross_walk['State'] = cross_walk['State'].map({'ACT ':'Australian Capital Territory',
                                               'NSW ':'New South Wales', 
                                               'WA ':'Western Australia', 
                                               'NT ':'Northern Territory',
                                               'Qld ':'Queensland', 
                                               'SA ':'South Australia', 
                                               'Tas. ':'Tasmania',
                                               'Vic. ':'Victoria'})
cross_walk['Electorial Dvision'] = cross_walk['Electorial Dvision'].str.rstrip()

# merge the crosswalk with the data
df_participation = pd.merge(df3, 
                    cross_walk, 
                    left_on = 'area', 
                    right_on = 'Electorial Dvision', 
                    how='inner', validate = 'm:1')

df_participation = df_participation.drop(['Electorial Dvision', 'Area (sq km)'], axis=1)

In [25]:
df_participation.head()

,area,statistic,age_category,quantity,State
0,Banks,Total participants,18-19 years,2355.0,New South Wales
1,Banks,Eligible participants,18-19 years,2945.0,New South Wales
2,Banks,Participation rate (%),18-19 years,80.0,New South Wales
3,Banks,Total participants,20-24 years,6361.0,New South Wales
4,Banks,Eligible participants,20-24 years,8319.0,New South Wales


### Step 4 : Abstraction of Participation results pull - Participation Rates

In [26]:
def pull_data_participation(file_name, table, cross_walk, columns, usecols):
    df = pd.read_excel(file_name, 
                      sheet_name=table, # obtain data from table 4 in the file
                      usecols=usecols, # the relevant columns are "A:S"
                      skiprows=6, # skip 7 rows so we start from row 8. **it doesn't specify stop?
                      names=columns # input a column name
                     )
    # Step 1 : Remove the unnecessary data that resides at the bottom of the spreadsheet
    df = df.drop(range(643, 651))
    df.tail()

    # Step 2 : Remove rows where it contains the observation "Divison" and "Australia" itself
    df1 = df[~df['area'].str.contains("Division|Australia$", na=False)].reset_index()
    df1 = df1.drop(range(632,635)).drop('index', axis=1)
    df1.tail()

    # Step 3 : Remove rows where it contains only NaN
    df2 = df1.dropna(how='all')
    df2 = df2.reset_index().drop('index', axis=1)
    df2.tail()

    # Step 4 : Forward fill the area observations
    df2['area'] = df2['area'].fillna(method='ffill')
    
    # Now, we need to "melt" our data
    df3 = pd.melt(df2, id_vars=['area', 'statistic'], var_name = 'age_category', value_name='quantity')

    # remove unnecessary totals, clean strings
    df3 = df3[~df3['area'].str.contains("(Total)", na=False)].reset_index().drop('index', axis=1)
    df3['area'] = df3['area'].str.replace(r"\([^)]*\)", "", regex=True)
    df3.tail()
    
    # merge the crosswalk with the data
    df_participation = pd.merge(df3, 
                        cross_walk, 
                        left_on = 'area', 
                        right_on = 'Electorial Dvision', 
                        how='inner', validate = 'm:1')

    df_participation = df_participation.drop(['Electorial Dvision', 'Area (sq km)'], axis=1)
    
    return df_participation

In [29]:
female_cols = ["area", 
               "statistic", 
               "18-19 years", 
               "20-24 years", 
               "25-29 years",
               "30-34 years",
               "35-39 years", 
               "40-44 years",    
               "45-49 years",
               "50-54 years",
               "55-59 years",
               "60-64 years",
               "65-69 years",
               "70-74 years",
               "75-79 years",
               "80-84 years",
               "85 years and over",
               "Total Females"]

# get female participation
female_participation = pull_data_participation(file_name, 'Table 6', cross_walk, female_cols,'A:R')
female_participation['gender'] = 'female'
female_participation.head()

<ipython-input-26-fba7b93cd65f>:67: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df3 = df3[~df3['area'].str.contains("(Total)", na=False)].reset_index().drop('index', axis=1)


,area,statistic,age_category,quantity,State,gender
0,Banks,Total participants,18-19 years,1253.0,New South Wales,female
1,Banks,Eligible participants,18-19 years,1514.0,New South Wales,female
2,Banks,Participation rate (%),18-19 years,82.8,New South Wales,female
3,Banks,Total participants,20-24 years,3263.0,New South Wales,female
4,Banks,Eligible participants,20-24 years,4095.0,New South Wales,female


In [36]:
male_cols = ["area", 
               "statistic", 
               "18-19 years", 
               "20-24 years", 
               "25-29 years",
               "30-34 years",
               "35-39 years", 
               "40-44 years",    
               "45-49 years",
               "50-54 years",
               "55-59 years",
               "60-64 years",
               "65-69 years",
               "70-74 years",
               "75-79 years",
               "80-84 years",
               "85 years and over",
               "Total Males"]

# get male participation
male_participation = pull_data_participation(file_name, 'Table 5', cross_walk, male_cols,'A:R')
male_participation['gender'] = 'male'
male_participation.head()

<ipython-input-26-fba7b93cd65f>:67: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df3 = df3[~df3['area'].str.contains("(Total)", na=False)].reset_index().drop('index', axis=1)


,area,statistic,age_category,quantity,State,gender
0,Banks,Total participants,18-19 years,1102.0,New South Wales,male
1,Banks,Eligible participants,18-19 years,1431.0,New South Wales,male
2,Banks,Participation rate (%),18-19 years,77.0,New South Wales,male
3,Banks,Total participants,20-24 years,3098.0,New South Wales,male
4,Banks,Eligible participants,20-24 years,4224.0,New South Wales,male


In [37]:
# get participation total. Display how both genders are represented here
participation_by_gender = pd.concat([female_participation, male_participation], axis=0)
participation_by_gender = participation_by_gender.reset_index().drop('index', axis=1)
participation_by_gender[(participation_by_gender['area']=='Banks') & 
                        (participation_by_gender['age_category']=='18-19 years')]

,area,statistic,age_category,quantity,State,gender
0,Banks,Total participants,18-19 years,1253.0,New South Wales,female
1,Banks,Eligible participants,18-19 years,1514.0,New South Wales,female
2,Banks,Participation rate (%),18-19 years,82.8,New South Wales,female
7200,Banks,Total participants,18-19 years,1102.0,New South Wales,male
7201,Banks,Eligible participants,18-19 years,1431.0,New South Wales,male
7202,Banks,Participation rate (%),18-19 years,77.0,New South Wales,male


### Data Cleaning (AMLSR)

* AMLSR's Layout is different. Hence, the code is not as long winded as before

In [38]:
# file name and read data, slightly different than participation rates but simpler
file_name_response = 'australian_marriage_law_survey_response.xls'
response_df = pd.read_excel(file_name_response, 
                            sheet_name='Table 2', # obtain data from table 4 in the file
                            usecols="A:F", # the relevant columns are "A:S"
                            skiprows=6, # skip 7 rows so we start from row 8. **it doesn't specify stop?
                            names=["area", 
                                 'vote_yes_qty', 
                                 "vote_yes_pct",
                                 "vote_no_qty",
                                 "vote_no_pct",
                                 "vote_total_qty"])
response_df = response_df.drop(range(173, 184))
response_df.tail()

,area,vote_yes_qty,vote_yes_pct,vote_no_qty,vote_no_pct,vote_total_qty
168,NaN,NaN,NaN,NaN,NaN,NaN
169,Australian Capital Territory Divisions,NaN,NaN,NaN,NaN,NaN
170,Canberra(d),89590.0,74.1,31361.0,25.9,120951.0
171,Fenner(e),85869.0,74.0,30159.0,26.0,116028.0
172,Australian Capital Territory (Total),175459.0,74.0,61520.0,26.0,236979.0


In [40]:
# remove all the unnecessary parts and the full NaN rows. Most of the procedures highlighted above
# are the same but has just been made more concise.
response_df1 = (response_df[~response_df['area']
                            .str.contains("Division|Australia$", na=False)]
                            .reset_index()
                            .drop('index', axis=1)
                            .dropna(how='all')
                            .reset_index()
                            .drop('index', axis=1))
response_df1.head()

,area,vote_yes_qty,vote_yes_pct,vote_no_qty,vote_no_pct,vote_total_qty
0,Banks,37736.0,44.9,46343.0,55.1,84079.0
1,Barton,37153.0,43.6,47984.0,56.4,85137.0
2,Bennelong,42943.0,49.8,43215.0,50.2,86158.0
3,Berowra,48471.0,54.6,40369.0,45.4,88840.0
4,Blaxland,20406.0,26.1,57926.0,73.9,78332.0


In [41]:
# remove the unnecessary totals
response_df1 = (response_df1[~response_df1['area'].str.contains("(Total)", na=False)]
                .reset_index()
                .drop('index', axis=1))

# remove the parantheses in some of the observations
response_df1['area'] = response_df1['area'].str.replace(r"\([^)]*\)", "", regex=True)
response_df1.tail()

<ipython-input-41-91ba139b03c2>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  response_df1 = (response_df1[~response_df1['area'].str.contains("(Total)", na=False)]


,area,vote_yes_qty,vote_yes_pct,vote_no_qty,vote_no_pct,vote_total_qty
145,Lyons,35894.0,58.7,25258.0,41.3,61152.0
146,Lingiari,19026.0,54.5,15898.0,45.5,34924.0
147,Solomon,29660.0,65.3,15792.0,34.7,45452.0
148,Canberra,89590.0,74.1,31361.0,25.9,120951.0
149,Fenner,85869.0,74.0,30159.0,26.0,116028.0


In [42]:
# merge the crosswalk with the data
df_response = pd.merge(response_df1, 
                       cross_walk, 
                       left_on = 'area', 
                       right_on = 'Electorial Dvision', 
                       how='inner', 
                       validate = '1:1')

# Final Data
df_response = df_response.drop(['Electorial Dvision', 'Area (sq km)'], axis=1)
df_response.head()

,area,vote_yes_qty,vote_yes_pct,vote_no_qty,vote_no_pct,vote_total_qty,State
0,Banks,37736.0,44.9,46343.0,55.1,84079.0,New South Wales
1,Barton,37153.0,43.6,47984.0,56.4,85137.0,New South Wales
2,Bennelong,42943.0,49.8,43215.0,50.2,86158.0,New South Wales
3,Berowra,48471.0,54.6,40369.0,45.4,88840.0,New South Wales
4,Blaxland,20406.0,26.1,57926.0,73.9,78332.0,New South Wales


# Conclusion :

* All DataFrames Obtained by this notebook are used in subsequent Notebooks - Analysis & Transferring to SQL